In [95]:
import scipy
import os
import numpy as np
import time
from IPython.display import clear_output

In [96]:
def loadFilePaths(filepath):
    files = []
    for path, subdirs, f in os.walk(filepath):
        for file in f:
            files.append(os.path.join(path, file))
    return files

In [97]:
DATA_DIR = r"C:\Users\Scott\Documents\Arduino\libraries\ACOMM\transducer\tests\kalman_tests\2022-08-18"
files = loadFilePaths(DATA_DIR)
files = [f for f in files if "note" not in f]
files = [f for f in files if "pingData" not in f.split('\\')[-1]]
adc_values = 4096
print(files)

['C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\10_847800_b_25khz_36khz_b__pingHz_1_L_200.txt', 'C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\11_847889_b_25khz_36khz_b__pingHz_1_L_200.txt', 'C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\12_847957_b_25khz_36khz_b__pingHz_1_L_200.txt', 'C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\13_848182_b_25khz_36khz_b__pingHz_4_straight_200.txt', 'C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\14_848224_b_25khz_36khz_b__pingHz_4_straight_200.txt', 'C:\\Users\\Scott\\Documents\\Arduino\\libraries\\ACOMM\\transducer\\tests\\kalman_tests\\2022-08-18\\beaconpos1\\15_848264_b_25khz_36khz_b__pingHz_4_straight_200.txt', 'C:\\U

In [98]:
def loadData(filepath):
    with open(filepath) as f:
        lines = f.readlines()
        return lines

In [99]:
filedata = [[(float(k.split(' ')[1])-adc_values/2)/(adc_values/2) for k in loadData(f)[100:-100]] for f in files]

In [100]:
filedata = [np.array(f) for f in filedata]

In [101]:
def get_indices_of_frequencies_between(lowerFreq, upperFreq, arr):
    indexOfLowerFreq = np.nonzero(arr < upperFreq)
    indexOfUpperFreq = np.nonzero(arr > lowerFreq)
    return np.intersect1d(indexOfLowerFreq,indexOfUpperFreq)

In [102]:

from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
N = 200
# sample spacing
T = 0.000005
x = np.linspace(0.0, N*T, N, endpoint=False)
trueMax1 = 0
trueMax2 = 0
xf = fftfreq(N, T)[:N//2]

beaconFreq = [25000, 36000]
beaconMaxValuePerStep = [0]*len(beaconFreq)
beaconData = []
for beacon in beaconFreq:
    beaconData.append({'indices': get_indices_of_frequencies_between(beacon-1000,beacon+1000,xf),
                    'triggerThreshold': 10,
                    'resetThreshold': 0.2,
                    'active': False})
allPingData = []
for data in filedata:
    pingData = []
    for i in range(len(data)-N):
        yf = fft(data[i:i+N])
        
        # get the max value of the frequency bins representing the beacons
        for index in range(len(beaconMaxValuePerStep)):
            beaconMaxVal = max([np.abs(yf[j]) for j in beaconData[index]['indices']])
            if beaconData[index]['active']:
                if beaconMaxVal < beaconData[index]['resetThreshold']:
                    beaconData[index]['active'] = False
            else:
                if beaconMaxVal >= beaconData[index]['triggerThreshold']:
                    beaconData[index]['active'] = True
                    pingData.append(str(index) + '_' + str(beaconFreq[index]) + '_' + str(i*0.000005))
        # clear_output(wait=True)
        # plt.plot(xf, 2.0/N * np.abs(yf[0:N//2]))
        # plt.grid()
        # plt.show()
    allPingData.append(pingData)

In [103]:
for dataIndex in range(len(allPingData)):
    allPingData[dataIndex] = allPingData[dataIndex][4:]

In [104]:
def saveData(data, filepath):
    with open(filepath, 'w') as f:
        f.write('\n'.join(data))

In [105]:
def extractFileName(filepath):
    return (filepath.split("\\")[-1]).split('.')[0]

def extractFilePath(filepath):
    return "\\".join(filepath.split('\\')[:-1]) + "\\"

def appendNameAndFileType(filepath):
    return str('pingData_') + filepath + '.txt'

saveFilePaths = [extractFilePath(filepath) + appendNameAndFileType(extractFileName(filepath)) for filepath in files]

In [106]:
for index in range(len(saveFilePaths)):
    saveData(allPingData[index], saveFilePaths[index])